In [22]:
"""

record inf time of different model loaders

exllama
wget https://github.com/turboderp/exllamav2/releases/download/v0.0.14/exllamav2-0.0.14+cu121-cp310-cp310-linux_x86_64.whl
pip install -q exllamav2-0.0.14+cu121-cp310-cp310-linux_x86_64.whl

"""

# Autocompletion
%config Completer.use_jedi = False

# Autoreload
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/workspace/llmsearch')

import gc
import torch
import ctypes

import nltk
import math
import torch
import random
import evaluate
import datasets
import langchain
import numpy as np
import transformers
from transformers.modeling_outputs import CausalLMOutputWithPast
from transformers import PreTrainedModel, PretrainedConfig, GenerationConfig, StoppingCriteria, AutoTokenizer, StoppingCriteriaList, AutoModel, AutoModelForCausalLM

import os
import gc
import ctypes
import traceback
from pathlib import Path
from typing import Any, Dict, Optional, Union, List

import time
import textwrap
from tqdm.auto import tqdm

from exllamav2 import (
    ExLlamaV2,
    ExLlamaV2Cache,
    ExLlamaV2Cache_8bit,
    ExLlamaV2Config
)

from datasets import load_dataset
from llmsearch.model_downloader import download_model_from_hf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
gsm8k_dataset = load_dataset("gsm8k", 'main')

torch.__version__

'2.2.0+cu121'

In [3]:
gsm8k_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})

In [4]:

def seed_everything(seed):
    """Seed for reproducibilty"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

class SingleTokenStoppingCriteria(StoppingCriteria):
    """End generation if end token is encountered
    does not support batched implementation yet"""

    def __init__(self, token_id):
      super().__init__()
      self.token_id =  token_id

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        res = []

        last_token_id = input_ids[0][-1]
        if last_token_id == self.token_id:
            return True
        return False


def cm():
    gc.collect()
    ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()

def seed_everything(seed):
    """Seed for reproducibilty"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False



def perform_single_example_inference(example, model, tokenizer,gen_kwargs):

    tokenized_input = tokenizer(example, return_tensors = "pt", add_special_tokens = False)
    tokenized_input['input_ids'] = tokenized_input['input_ids'].to('cuda:0')

    tokenized_input['attention_mask'] = tokenized_input['attention_mask'].to('cuda:0')
    # tokenized_input.to(device)
    # print(tokenized_input)

    model_out = model.generate(**tokenized_input, **gen_kwargs)
    prompt_tokens = len(tokenized_input['input_ids'][0])
    print(f"Prompt tokens - {prompt_tokens}")
    # print(model_out.tolist()[0])

    output_token_ids = model_out.tolist()[0]
    decoded_output = tokenizer.decode(output_token_ids, spaces_between_special_tokens = False)

    print(decoded_output)
    completion_tokens = len(output_token_ids) - prompt_tokens

    print(f"Completion Tokens - {completion_tokens}")

    return decoded_output, prompt_tokens, completion_tokens

In [35]:
# loaders

# exllama 2 backend loader
class Exllamav2HF(PreTrainedModel):
    def __init__(self, config: ExLlamaV2Config):
        super().__init__(PretrainedConfig())
        self.ex_config = config
        self.ex_model = ExLlamaV2(config)
        split = None
        if shared.args.gpu_split:
            split = [float(alloc) for alloc in shared.args.gpu_split.split(",")]

        self.ex_model.load(split)
        self.generation_config = GenerationConfig()
        self.loras = None

        if shared.args.cache_8bit:
            self.ex_cache = ExLlamaV2Cache_8bit(self.ex_model)
        else:
            self.ex_cache = ExLlamaV2Cache(self.ex_model)

        self.past_seq = None
        if shared.args.cfg_cache:
            if shared.args.cache_8bit:
                self.ex_cache_negative = ExLlamaV2Cache_8bit(self.ex_model)
            else:
                self.ex_cache_negative = ExLlamaV2Cache(self.ex_model)

            self.past_seq_negative = None

    def _validate_model_class(self):
        pass

    def _validate_model_kwargs(self, model_kwargs: Dict[str, Any]):
        pass

    def prepare_inputs_for_generation(self, input_ids, **kwargs):
        return {'input_ids': input_ids, **kwargs}

    @property
    def device(self) -> torch.device:
        return torch.device(0)

    def __call__(self, *args, **kwargs):
        use_cache = kwargs.get('use_cache', True)
        labels = kwargs.get('labels', None)
        past_key_values = kwargs.get('past_key_values', None)

        if len(args) > 0:
            if not shared.args.cfg_cache:
                print("Please enable the cfg-cache option to use CFG with ExLlamav2_HF.")
                return

            input_ids = args[0]
            is_negative = True
            past_seq = self.past_seq_negative
            ex_cache = self.ex_cache_negative
        else:
            input_ids = kwargs['input_ids']
            is_negative = False
            past_seq = self.past_seq
            ex_cache = self.ex_cache

        seq = input_ids[0].tolist()
        if is_negative and past_key_values is not None:
            seq = past_key_values + seq

        seq_tensor = torch.tensor(seq)
        reset = True

        # Make the forward call
        if labels is None:
            if past_seq is not None:
                min_length = min(past_seq.shape[0], seq_tensor.shape[0])
                indices = torch.nonzero(~torch.eq(past_seq[:min_length], seq_tensor[:min_length]))
                if len(indices) > 0:
                    longest_prefix = indices[0].item()
                else:
                    longest_prefix = min_length

                if longest_prefix > 0:
                    reset = False
                    ex_cache.current_seq_len = longest_prefix
                    if len(seq_tensor) - longest_prefix > 1:
                        self.ex_model.forward(seq_tensor[longest_prefix:-1].view(1, -1), ex_cache, preprocess_only=True, loras=self.loras)
                    elif len(seq_tensor) == longest_prefix:
                        # Very tricky: if the prefix we are reusing *is* the input_ids, then we have to back up the cache pointer by one,
                        # because we feed input_ids[-1] to forward() below, but that last token is already in the cache!
                        ex_cache.current_seq_len -= 1

            if reset:
                ex_cache.current_seq_len = 0
                if len(seq_tensor) > 1:
                    self.ex_model.forward(seq_tensor[:-1].view(1, -1), ex_cache, preprocess_only=True, loras=self.loras)

            logits = self.ex_model.forward(seq_tensor[-1:].view(1, -1), ex_cache, loras=self.loras).to(input_ids.device).float()
        else:
            ex_cache.current_seq_len = 0
            logits = self.ex_model.forward(seq_tensor.view(1, -1), ex_cache, last_id_only=False, loras=self.loras).float()

        if is_negative:
            self.past_seq_negative = seq_tensor
        else:
            self.past_seq = seq_tensor

        loss = None
        if labels is not None:
            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # Flatten the tokens
            loss_fct = CrossEntropyLoss()
            shift_logits = shift_logits.view(-1, logits.shape[-1])
            shift_labels = shift_labels.view(-1)
            # Enable model parallelism
            shift_labels = shift_labels.to(shift_logits.device)
            loss = loss_fct(shift_logits, shift_labels)

        return CausalLMOutputWithPast(logits=logits, past_key_values=seq if use_cache else None, loss=loss)

    @classmethod
    def from_pretrained(cls, pretrained_model_name_or_path: Optional[Union[str, os.PathLike]], *model_args, **kwargs):
        assert len(model_args) == 0 and len(kwargs) == 0, "extra args is currently not supported"
        if isinstance(pretrained_model_name_or_path, str):
            pretrained_model_name_or_path = Path(pretrained_model_name_or_path)


        config = ExLlamaV2Config()
        config.model_dir = str(pretrained_model_name_or_path)
        config.prepare()

        config.max_seq_len = shared.args.max_seq_len
        config.scale_pos_emb = shared.args.compress_pos_emb
        config.scale_alpha_value = shared.args.alpha_value
        config.no_flash_attn = shared.args.no_flash_attn

        return Exllamav2HF(config)

class Shared:
    class Args:
        def __init__(self):
            self.gpu_split = None

    def __init__(self):
        self.args = Shared.Args()

shared = Shared()
shared.args.gpu_split = None
shared.args.cache_8bit = None
shared.args.cfg_cache = None
# shared.args.model_dir = "/kaggle/input/"
shared.args.max_seq_len = 2048
shared.args.compress_pos_emb = 1
shared.args.alpha_value = 1
shared.args.no_flash_attn = 1


def load_model_with_exllama_2_hf_backend(model_loader_kwargs, tokenizer_kwargs):
    model = Exllamav2HF.from_pretrained(model_loader_kwargs['pretrained_model_name_or_path'])

    tokenizer = AutoTokenizer.from_pretrained(**tokenizer_kwargs, local_files_only=True)

    # make this dynamic
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

def load_model_with_hf_backend(model_loader_kwargs, tokenizer_kwargs):
    model = AutoModelForCausalLM.from_pretrained(**model_loader_kwargs, local_files_only=True)
    tokenizer = AutoTokenizer.from_pretrained(**tokenizer_kwargs, local_files_only=True)

    # make this dynamic
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

model_loader_backend_map = {
    "exllama_2_hf": load_model_with_exllama_2_hf_backend,
    "hf": load_model_with_hf_backend
}

def preprocess_dataset(dataset, tokenizer, encoding_kwargs, decoding_kwargs, pt, pt_cols, system_prompt, add_generation_prompt = True):

    def wrapper(sample):
        """Takes in a sample, formats it using prompt template, applies chat template and returns the formatted string"""
        messages = [] if system_prompt is None else [{"role": "system", "content": system_prompt}]
        formatted_pt = pt.format(**{pt_col : sample[pt_col] for pt_col in pt_cols})
        messages.append(
            {
                "role": "user",
                "content": formatted_pt,
            }
        )
        formatted_pt_with_ct = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt=add_generation_prompt)
        return formatted_pt_with_ct

    def actual_input(sample):
        """Takes in a sample, formats it using prompt template, applies chat template and returns the formatted string"""
        return sample[pt_cols[0]]



    pt_dataset = dataset.map(
        lambda sample : {
            "X" : wrapper(sample),
            'actual input' : actual_input(sample),
        }
    )

    return pt_dataset

from llmsearch.utils.model_utils import batcher, decoder_parser

def perform_inference_batched(model, tokenizer, model_inputs, gen_kwargs, tokenizer_encoding_kwargs,tokenizer_decoding_kwargs, batch_size, seed):
    batch_latency = []
    batch_shape = []
    outputs = []



    seed_everything(seed)

    for batch in tqdm(batcher(iterable = model_inputs, batch_size = batch_size), total = math.ceil(len(model_inputs) / batch_size)):
        model_input = [f"{x['X']}" for x in batch]

        encoded_input = tokenizer(text = model_input, **tokenizer_encoding_kwargs, return_tensors = "pt")

        input_ids = encoded_input['input_ids'].to('cuda:0')
        attention_mask = encoded_input['attention_mask'].to('cuda:0')

        batch_shape.append(input_ids.shape)

        start = time.time()
        output_ids = model.generate(input_ids, attention_mask = attention_mask, **gen_kwargs)

        # print(f"output ids - {output_ids.shape}")

        end = time.time()

        latency = end - start

        decoded_output = tokenizer.batch_decode(output_ids, **tokenizer_decoding_kwargs)

        # remove prompt
        decoded_output = decoder_parser(outputs = decoded_output, formatted_prompts = model_inputs, prepoc = lambda x : x.strip())

        batch_latency.append(latency)

        print(f"input - {model_input}, output - {decoded_output}, latency - {latency}, shape - {input_ids.shape}")

        # add tps

        outputs.extend(decoded_output)

    return outputs, batch_latency, batch_shape




def benchmark_model(model_loader_kwargs, tokenizer_loader_kwargs, model_id, model_backend, tokenizer_encoding_kwargs,
                    tokenizer_decoding_kwargs, dataset, pt, pt_cols, system_prompt, gen_kwargs, add_generation_prompt = True, batch_size = 1, seed = 42, bm_sample_size = 20, model_branch = "main"):

    cm()

    artifacts = {}

    benchmark_dict = {
        'model_id' : model_id,
        'model_loader_kwargs' : model_loader_kwargs,
        'tokenizer_loader_kwargs' : tokenizer_loader_kwargs,
        'model_backend' : model_backend,
        'model_branch' : model_branch,
        'tokenizer_encoding_kwargs' : tokenizer_encoding_kwargs,
        'tokenizer_decoding_kwargs' : tokenizer_decoding_kwargs,
        'pt' : pt,
        'pt_cols' : pt_cols,
        'system_prompt' : system_prompt,
        'batch_size' : batch_size,
        'seed' : seed,
        'bm_sample_size' : bm_sample_size,
        'gen_kwargs' : gen_kwargs,
    }


    # 1. download model
    temp_model_dir = Path(f"/workspace/temp_model_dir/")
    temp_model_dir.mkdir(exist_ok = True, parents = True)
    output_folder = download_model_from_hf(model_id, save_dir = temp_model_dir, branch = model_branch)
    model_loader_kwargs['pretrained_model_name_or_path'] = output_folder
    tokenizer_loader_kwargs['pretrained_model_name_or_path'] = output_folder

    # 2. load model
    start = time.time()
    model, tokenizer = model_loader_backend_map[model_backend](model_loader_kwargs, tokenizer_loader_kwargs)
    end = time.time()

    artifacts['model'] = model
    artifacts['tokenizer'] = tokenizer

    # 3. process datset
    processed_dataset = preprocess_dataset(dataset, tokenizer, tokenizer_encoding_kwargs, tokenizer_decoding_kwargs, pt, pt_cols, system_prompt = system_prompt, add_generation_prompt = add_generation_prompt)

    # show 2 samples of processed dataset
    print("Processed Dataset:\n")
    for i in range(2):
        print(processed_dataset[i]['X'])
        print('\n')
        print('---' * 10)
        print('\n')

    model_loading_time = end - start
    benchmark_dict['model_loading_time'] = model_loading_time

    bm_samples = processed_dataset.shuffle(seed = seed).select(range(bm_sample_size))

    # 4. perform inference
    outputs, batch_latency, batch_shape = perform_inference_batched(model, tokenizer, bm_samples, gen_kwargs, tokenizer_encoding_kwargs, tokenizer_decoding_kwargs, batch_size, seed)

    benchmark_dict['model_out'] = {
        'outputs' : outputs,
        'batch_latency' : batch_latency,
        'batch_shape' : batch_shape,
    }

    return artifacts, benchmark_dict


model_loader_kwargs = {
    'device_map' : {'' : 0}
}
tokenizer_loader_kwargs = {
    'use_fast' : False,
    'legacy' : False,
}
tokenizer_encoding_kwargs = {
    # pad to longest seq in batch
    'padding' : True,
}
tokenizer_decoding_kwargs = {}

model_id = "TheBloke/CapybaraHermes-2.5-Mistral-7B-GPTQ"
model_backend = "exllama_2_hf"
dataset = gsm8k_dataset['train']
pt = textwrap.dedent("""\
    Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
    A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.

    Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
    A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.

    Q: {question}""")
pt_cols = ['question']
system_prompt = "Solve the following math problems."
batch_size = 1

gen_kwargs = {
    'max_new_tokens' : 500,

}
bm_sample_size = 20



res = benchmark_model(
    model_loader_kwargs=model_loader_kwargs,
    tokenizer_loader_kwargs=tokenizer_loader_kwargs,
    model_id=model_id,
    model_backend=model_backend,
    dataset = dataset,
    tokenizer_encoding_kwargs=tokenizer_encoding_kwargs,
    tokenizer_decoding_kwargs=tokenizer_decoding_kwargs,
    pt=pt,
    pt_cols=pt_cols,
    system_prompt=system_prompt,
    gen_kwargs=gen_kwargs,
    batch_size=batch_size,
    bm_sample_size=bm_sample_size,
)

Model already exists in /workspace/temp_model_dir/TheBloke_CapybaraHermes-2.5-Mistral-7B-GPTQ. Checking the model files...
Checksum validated: model.safetensors  fc7d5419e6d124db8bd07a4c3332f867819dbde179db39e83611f4f7fcf23c3a
Checksum validated: tokenizer.model  dadfd56d766715c61d2ef780a525ab43b8e6da4de6865bda3d95fdef5e134055
[+] Validated checksums of all model files!


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Processed Dataset:

<|im_start|>system
Solve the following math problems.<|im_end|>
<|im_start|>user
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.

Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.

Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?<|im_end|>
<|im_start|>assistant



------------------------------


<|im_start|>system
Solve the following math problems.<|im_end|>
<|im_start|>user
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove 

  0%|          | 0/20 [00:00<?, ?it/s]

input - ['<|im_start|>system\nSolve the following math problems.<|im_end|>\n<|im_start|>user\nQ: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nA: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.\n\nQ: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?\nA: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.\n\nQ: Mimi picked up 2 dozen seashells on the beach.  Kyle found twice as many shells as Mimi and put them in his pocket. Leigh grabbed one-third of the shells that Kyle found.  How many seashells did Leigh have?<|im_end|>\n<|im_start|>assistant\n'], output - ['<|im_start|> system\nSolve the following math problems.<|im_end|>\n <|im_start|> user\nQ: There are 15 trees in the grove. Grove w

In [14]:
res['tokenizer'].max_length

AttributeError: 'LlamaTokenizer' object has no attribute 'max_length'

In [18]:
gsm8k_dataset

NameError: name 'gsm8k_dataset' is not defined

In [5]:
def load_dataset(samples = 10)
    sampled_dataset = gsm8k_dataset['train'].shuffle(seed=42).select(range(samples))
    return sampled_dataset



In [7]:







def load_model_with_hf_backend(model_id, loading_args, tokenizer_args):
    # model = AutoModelForCausalLM.from_pretrained(model_id, **loading_args)
    # tokenizer = AutoTokenizer.from_pretrained(model_id, *tokenizer_args)

    model = Exllamav2HF.from_pretrained(pretrained_model_name_or_path = model_id)
    tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False, legacy=False)

    return model, tokenizer

def prepare_input(tokenizer,question):
    text = textwrap.dedent("""\
    Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
    A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.

    Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
    A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.

    Q: {question}""")

    pt = langchain.PromptTemplate.from_template(text)
    formatted_pt = pt.format(question=question)

    messages = [
        {
            "role": "system",
            "content": "You are a friendly assistant who can solve math problems",
        },
        {"role": "user", "content": formatted_pt},
    ]

    ct_sample = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt=True)
    return ct_sample


def driver(model_id, loading_args, tokenizer_args, gen_params1):

    cm()

    print(model_id)
    print(f"Model loading args - {loading_args}")
    print(f"tokenizer args - {tokenizer_args}\n")

    start = time.time()
    model, tokenizer = load_model_with_hf_backend(model_id, loading_args, tokenizer_args)
    loading_time = (time.time() - start)


    avg_tps = []

    seed = 42
    seed_everything(seed)

    detail_dict = {
        'model_loading_time' : loading_time,
        'input_args' : {
            'model_id' : model_id,
            'loading_args' : loading_args,
            'tokenizer_args' : tokenizer_args,
            'seed' : seed,
        },
        'out' : [],
        'inf_latency' : [],
        'p_tokens' : [],
        'c_tokens' : [],
        'tps' : [],
    }


    for idx, item in tqdm(enumerate(sampled_dataset)):
        prepared_example = prepare_input(tokenizer,item['question'])
        start = time.time()
        out, prompt_tokens, c_tokens = perform_single_example_inference(prepared_example, model,tokenizer, gen_params1)
        inf_latency = (time.time() - start)

        detail_dict['out'].append(out)
        detail_dict['inf_latency'].append(inf_latency)
        detail_dict['p_tokens'].append(prompt_tokens)
        detail_dict['c_tokens'].append(c_tokens)

        tps = c_tokens/inf_latency
        detail_dict['tps'].append(tps)

        # record post warmup (3 eg)
        if idx > 2:
            avg_tps.append(tps)
            print(f"idx  - {idx} , tps - {tps}\n")

            print('\n\n')
            print('----' * 10)
            print('\n\n')


    a_tps = sum(avg_tps)/len(avg_tps)
    print(f"Average tps - {a_tps}")

    print('\n', '----' * 15, '\n\n')

    detail_dict['average_tps'] = a_tps

    return detail_dict



In [8]:
stopping_criteria = StoppingCriteriaList([SingleTokenStoppingCriteria(token_id=32000)])
model_id = "/workspace/capybarahermes-2.5-gptq/TheBloke_CapybaraHermes-2.5-Mistral-7B-GPTQ"
loading_args = {
    'device_map' : {"" : 0},
}
tokenizer_args = {

}
gen_params1 = {
    'max_new_tokens' : 500,
    'stopping_criteria' : stopping_criteria,
    'do_sample' : False,
}
detail_dict = driver(model_id, loading_args, tokenizer_args, gen_params1)

print('\n\n')
print('****' * 20)



/workspace/capybarahermes-2.5-gptq/TheBloke_CapybaraHermes-2.5-Mistral-7B-GPTQ
Model loading args - {'device_map': {'': 0}}
tokenizer args - {}



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


0it [00:00, ?it/s]

Prompt tokens - 242
<|im_start|>system
You are a friendly assistant who can solve math problems<|im_end|>
<|im_start|>user
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.

Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.

Q: Mimi picked up 2 dozen seashells on the beach.  Kyle found twice as many shells as Mimi and put them in his pocket. Leigh grabbed one-third of the shells that Kyle found.  How many seashells did Leigh have?<|im_end|>
<|im_start|>assistant
A: Mimi picked up 2 dozen seashells, which is 2 * 12 = 24 seashells.
Kyle found twice as many shells as Mimi, so he found 24

In [9]:
detail_dict

{'model_loading_time': 1.8519587516784668,
 'input_args': {'model_id': '/workspace/capybarahermes-2.5-gptq/TheBloke_CapybaraHermes-2.5-Mistral-7B-GPTQ',
  'loading_args': {'device_map': {'': 0}},
  'tokenizer_args': {},
  'seed': 42},
 'out': ['<|im_start|>system\nYou are a friendly assistant who can solve math problems<|im_end|>\n<|im_start|>user\nQ: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nA: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.\n\nQ: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?\nA: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.\n\nQ: Mimi picked up 2 dozen seashells on the beach.  Kyle found twice as many shells as Mimi and put them in his pocket. Leigh grabbed

In [10]:
import json

def json_dump(ob : dict, file_path: Path):
    with open(file_path, 'w', encoding="utf-8") as json_file:
        json.dump(ob, json_file, indent=4)

idx = 1
file_path = f"./{model_id.split('/')[-1]}-{idx}-inf-time.json"
json_dump(detail_dict, file_path)